In [1]:
# # VacationPy
# ----

# #### Note
# * Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

# * Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os
import sys
from pprint import pprint

In [3]:
# Import API key
from config import gkey

#print(gkey)

In [4]:
### Store Part I results into DataFrame
#* Load the csv exported in Part I to a DataFrame

In [5]:
city_data = pd.read_csv('city_list_data.csv')

city_data.head()

,Unnamed: 0,Name,Country,lat,long,Real Lat,Real Long,Max temp,Humidity,Cloudiness,Wind Speed,Date Rec
0,0,lezajsk,pl,50.36,22.28,50.26,22.42,48.20,76.0,100.0,11.41,1603043626
1,2,burla,ru,53.10,78.14,21.50,83.87,77.72,82.0,100.0,1.63,1603043626
2,3,east london,za,-72.97,50.18,-33.02,27.91,62.60,72.0,50.0,10.29,1603043626
3,4,bredasdorp,za,-72.42,22.52,-34.53,20.04,62.60,72.0,66.0,5.82,1603043626
4,5,dikson,ru,88.25,91.17,73.51,80.55,26.53,89.0,81.0,29.82,1603043626


In [6]:
city_data.dtypes

Unnamed: 0      int64
Name           object
Country        object
lat           float64
long          float64
Real Lat      float64
Real Long     float64
Max temp      float64
Humidity      float64
Cloudiness    float64
Wind Speed    float64
Date Rec        int64
dtype: object

In [7]:
# ### Humidity Heatmap
# * Configure gmaps.
# * Use the Lat and Lng as locations and Humidity as the weight.
# * Add Heatmap layer to map.

In [8]:
# Configure gmaps
gmaps.configure(api_key=gkey)
m = gmaps.figure(center=(0,30), zoom_level=1.75)

In [9]:
# loading the city data and humidity as the weight parameter
fig_loc = city_data[['Real Lat','Real Long']]
fig_loc
weights = city_data['Humidity']
print(weights)



0      76.0
1      82.0
2      72.0
3      72.0
4      89.0
       ... 
554    93.0
555    93.0
556    63.0
557    52.0
558    69.0
Name: Humidity, Length: 559, dtype: float64


In [10]:
# configure map layout
# m = gmaps.figure(map_type='HYBRID', layout={
#         'width': '700px',
#         'height': '525px',
#         'padding': '3px',
#         'border': '1px solid black'
# })

In [11]:
# add heatmap layer to gmap

m.add_layer(gmaps.heatmap_layer(fig_loc, weights=weights, point_radius= 5, max_intensity=100))


m



Figure(layout=FigureLayout(height='420px'))

In [12]:
### Create new DataFrame fitting weather criteria
# Narrow down the cities to fit weather conditions.
# Drop any rows will null values.

In [13]:
# only cities within the right temp range
new_df = city_data.loc[(city_data['Max temp']>70) & (city_data['Max temp']<80)]


In [14]:
# only cities with wind speed less than 10 mph
new_df = new_df.loc[(new_df ['Wind Speed']<10)]


In [15]:
# only cities with zero cloudiness
new_df = new_df .loc[(new_df ['Cloudiness']==0)]
new_df.head()


,Unnamed: 0,Name,Country,lat,long,Real Lat,Real Long,Max temp,Humidity,Cloudiness,Wind Speed,Date Rec
37,42,riyadh,sa,24.63,46.36,24.69,46.72,77.00,22.0,0.0,3.36,1603043531
52,57,general roca,ar,-37.18,-66.42,-39.03,-67.58,73.40,23.0,0.0,5.82,1603043630
304,330,khorramshahr,ir,30.40,46.68,30.44,48.17,78.80,31.0,0.0,4.70,1603043649
352,383,marzuq,ly,25.34,14.42,14.40,46.47,70.88,22.0,0.0,1.30,1603043654
412,447,dindori,in,23.11,80.77,22.95,81.08,71.06,80.0,0.0,2.95,1603043659


In [16]:
### Hotel Map
# * Store into variable named `hotel_df`.
# * Add a "Hotel Name" column to the DataFrame.
# * Set parameters to search for hotels with 5000 meters.
# * Hit the Google Places API for each city's coordinates.
# * Store the first Hotel result into the DataFrame.
# * Plot markers on top of the heatmap.


In [17]:
# store 'Hotel Name' as a column in the new hotel_df dataframe
hotel_df = pd.DataFrame(new_df)
hotel_name = []

# create the list variable with proper length
for row in range(len(hotel_df)):
    hotel_name.append('')

# add hotel name list into dataframe
hotel_df['Hotel Name'] = hotel_name
hotel_df= pd.DataFrame(hotel_df)
hotel_df

,Unnamed: 0,Name,Country,lat,long,Real Lat,Real Long,Max temp,Humidity,Cloudiness,Wind Speed,Date Rec,Hotel Name
37,42,riyadh,sa,24.63,46.36,24.69,46.72,77.00,22.0,0.0,3.36,1603043531,
52,57,general roca,ar,-37.18,-66.42,-39.03,-67.58,73.40,23.0,0.0,5.82,1603043630,
304,330,khorramshahr,ir,30.40,46.68,30.44,48.17,78.80,31.0,0.0,4.70,1603043649,
352,383,marzuq,ly,25.34,14.42,14.40,46.47,70.88,22.0,0.0,1.30,1603043654,
412,447,dindori,in,23.11,80.77,22.95,81.08,71.06,80.0,0.0,2.95,1603043659,


In [18]:
# parameters for Google api search

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'

params = { 
    
    #'rankby' : 'prominence',
    'radius': 5000,
    'types': 'lodging',
    'key' : gkey
}


In [19]:
# use iterrows to iterate through pandas dataframe
hotel_lat = []
hotel_long = []
hotel_name = []

for index, row in hotel_df.iterrows():
    lat = row['Real Lat']
    long = row['Real Long']
    
    params['location'] = f"{lat},{long}"

   
    # create URL and api request
    hotel_data = requests.get(base_url, params=params).json()
    # print (f'{base_url} {params}')
    
    print(json.dumps(hotel_data['results'], indent=6, sort_keys=True))
    
    # extract hotel name and loc
    #     hotel_lat.append(hotel_data['results'][0]['geometry']['location']['lat'])
    #     hotel_long.append(hotel_data['results'][0]['geometry']['location']['lng'])
    try: 
        hotel_name.append(hotel_data['results'][1]['name'])
        print(hotel_lat, hotel_long, hotel_name)
    except IndexError:
        hotel_name.append('No hotel')
        #print("No hotel")
        
    

[
      {
            "business_status": "OPERATIONAL",
            "geometry": {
                  "location": {
                        "lat": 24.666677,
                        "lng": 46.692159
                  },
                  "viewport": {
                        "northeast": {
                              "lat": 24.6677663802915,
                              "lng": 46.6937272802915
                        },
                        "southwest": {
                              "lat": 24.6650684197085,
                              "lng": 46.6910293197085
                        }
                  }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "InterContinental Riyadh",
            "opening_hours": {
                  "open_now": true
            },
            "photos": [
                  {
                        "height": 843,
                        "html_attributions": [
          

[
      {
            "business_status": "OPERATIONAL",
            "geometry": {
                  "location": {
                        "lat": -39.04000319999999,
                        "lng": -67.6124162
                  },
                  "viewport": {
                        "northeast": {
                              "lat": -39.0386537197085,
                              "lng": -67.61107076970848
                        },
                        "southwest": {
                              "lat": -39.0413516802915,
                              "lng": -67.6137687302915
                        }
                  }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Hotel El Recreo",
            "opening_hours": {
                  "open_now": true
            },
            "photos": [
                  {
                        "height": 678,
                        "html_attributions": [
  

[
      {
            "business_status": "OPERATIONAL",
            "geometry": {
                  "location": {
                        "lat": 30.43962270000001,
                        "lng": 48.1929586
                  },
                  "viewport": {
                        "northeast": {
                              "lat": 30.4409406802915,
                              "lng": 48.19430698029149
                        },
                        "southwest": {
                              "lat": 30.43824271970851,
                              "lng": 48.19160901970849
                        }
                  }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "\u062f\u0641\u062a\u0631 \u0646\u0645\u0627\u06cc\u0646\u062f\u06af\u06cc \u0634\u0631\u06a9\u062a \u0628\u0627\u0632\u0627\u0631\u06cc\u0627\u0628\u0627\u0646 \u0627\u06cc\u0631\u0627\u0646\u06cc\u0627\u0646 \u0632\u0645\u06cc\u0646BI

[]
[
      {
            "business_status": "OPERATIONAL",
            "geometry": {
                  "location": {
                        "lat": 22.94892,
                        "lng": 81.07915589999999
                  },
                  "viewport": {
                        "northeast": {
                              "lat": 22.9502269302915,
                              "lng": 81.0805383302915
                        },
                        "southwest": {
                              "lat": 22.9475289697085,
                              "lng": 81.0778403697085
                        }
                  }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Maa Reva Bhojnalay",
            "place_id": "ChIJ98NXLXkjhzkR_B-fVaR6-D8",
            "plus_code": {
                  "compound_code": "W3XH+HM Dindori, Madhya Pradesh, India",
                  "global_code": "7MJ3W3XH+HM"
          

In [22]:
# create hotels dataframe

hotel_df['Hotel Name'] = hotel_name

hotel_df

,Unnamed: 0,Name,Country,lat,long,Real Lat,Real Long,Max temp,Humidity,Cloudiness,Wind Speed,Date Rec,Hotel Name
37,42,riyadh,sa,24.63,46.36,24.69,46.72,77.00,22.0,0.0,3.36,1603043531,Hotel Al Khozama
52,57,general roca,ar,-37.18,-66.42,-39.03,-67.58,73.40,23.0,0.0,5.82,1603043630,Hotel Huemul
304,330,khorramshahr,ir,30.40,46.68,30.44,48.17,78.80,31.0,0.0,4.70,1603043649,مامور سرا مخابرات و مایکروویو
352,383,marzuq,ly,25.34,14.42,14.40,46.47,70.88,22.0,0.0,1.30,1603043654,No hotel
412,447,dindori,in,23.11,80.77,22.95,81.08,71.06,80.0,0.0,2.95,1603043659,SATGURU PHOTO STUDIO


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Real Lat", "Real Long"]]

In [24]:
# Add marker layer ontop of heat map
# Display figure
#print(hotel_info)

In [25]:
# Create desired location hotel symbol layer
# will include heat map and "no hotel around desired locations"
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)


m.add_layer(hotel_layer)

m

Figure(layout=FigureLayout(height='420px'))